# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Условия задачи:

1) Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).

2) При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

3) Бюджет на разработку скважин в регионе — 10 млрд рублей.

4) При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

5) После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Описание данных:

1) id — уникальный идентификатор скважины;

2) f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);

3) product — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [2]:
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_3 = pd.read_csv('/datasets/geo_data_2.csv')
data_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
data_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [4]:
data_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [5]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [9]:
data_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [10]:
data_3.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


На данный момент мы видим что в таблицах данные заполнены полностью, дополнительной предобработки не требуется. Также поскольку в условии задачи сказано что все показатели важны мы не можем ничего убрать или поменять. Похоже что по подготовке данных это все. Единственное будет необходимо убрать id из признаков, иначе модель не сможет обучиться.

Также хочу добавить что на первый взгляд мы видим что на первом и третьем месторождении у нас среднее значительно выше чем у второго, а поскольку кол-во строк в датасетах одинаковое, можно сделать вывод что в первом и третьем месторождении запасов больше, и их разработка более выгодна. Можно было бы еще попытаться сравнить другие параметры, но мы не знаем насколько хорошо или плохо то что они большие или маленькие, обратим внимание что у второго региона разброс параметров меньше чем у других, вероятно это скажется на точности модели. Проверим.

## Обучение и проверка модели

In [11]:
features_1 = data_1.drop(['id','product'], axis = 1)
target_1 = data_1['product']
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1, 
                                                                   test_size = 0.25, random_state=12345)

In [12]:
features_2 = data_2.drop(['id','product'], axis = 1)
target_2 = data_2['product']
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2, 
                                                                     test_size = 0.25, random_state=12345)

In [13]:
features_3 = data_3.drop(['id','product'], axis = 1)
target_3 = data_3['product']
features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(features_3, target_3, 
                                                                      test_size = 0.25, random_state=12345)

In [14]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predicted_valid_1 = model_1.predict(features_valid_1)
predicted_valid_1.mean()

92.59256778438038

In [15]:
rmse_1 = mean_squared_error(target_valid_1, predicted_valid_1) ** 0.5
rmse_1

37.5794217150813

In [16]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predicted_valid_2 = model_2.predict(features_valid_2)
predicted_valid_2.mean()

68.728546895446

In [17]:
rmse_2 = mean_squared_error(target_valid_2, predicted_valid_2) ** 0.5
rmse_2

0.893099286775616

In [18]:
model_3 = LinearRegression()
model_3.fit(features_train_3, target_train_3)
predicted_valid_3 = model_3.predict(features_valid_3)
predicted_valid_3.mean()

94.96504596800489

In [19]:
rmse_3 = mean_squared_error(target_valid_3, predicted_valid_3) ** 0.5
rmse_3

40.02970873393434

Мы видим что в первом и третьем регионе модель предсказывает большее количество сырья чем во втором, но при этом среднее квадратичное отклонение в них тоже выше. Для того чтобы принять решение, какой регион лучше подходит для разработки требуется посчитать прибыль в деньгах и оценить риски, чем мы сейчас и займемся.

На данный момент я предполагаю что первый и третий регион будут выгоднее для разработки, потому что среднее квадратичное отклонение подразумевает, что отклонение будет и в большую и в меньшую сторону, т.е. в них в среднем будет от 55 до примерно 130 тысяч баррелей против разброса 68 - 70 тысяч баррелей во втором регионе, что в свою очередь в худшем случае меньше всего на 20-25%, а в лучшем больше почти в два раза. Пока моим фаворитом является третий регион.

## Подготовка к расчёту прибыли

Для начала обозначим константы, уменьшив суммы на "000", чтобы было проще считать. Далее посчитаем сколько необходимо добыть тысяч баррелей, чтобы как минимум не терпеть убытки. С учетом что по условиям задачи у нас бюджет 1 млрд и необходимо выбрать 200 точек, предположим что стоимость разработки одной скважины 5 млн рублей, и в эту сумму входит себистоимость\аммортизация оборудования, налоги и найм рабочих.

In [20]:
investition = 10000000
price_for_kbarrel = 450
number_points = 200
price_for_point = investition / number_points

In [21]:
need_oil_total = investition / price_for_kbarrel
need_oil_total

22222.222222222223

In [22]:
need_oil_per_point = (price_for_point) / price_for_kbarrel
need_oil_per_point

111.11111111111111

Итого, для того чтобы добыча нефти из скважины не была убыточна, достаточно всего 13 тысяч бареллей нефти, и в принципе, судя по среднему которое мы нашли, все три региона будут являться прибыльными для добычи. Раз мы выяснили что прибыльны будут все, то давайте посчитаем какую прибыль нам принесут регионы если пользоваться предсказаниями модели.

In [23]:
def value_on_region(predict, answer):
    predict_series = pd.Series(predict)
    answer_reset = answer.reset_index().drop('index', axis = 1)
    sort_predict = predict_series.sort_values(ascending = False)
    sort_answer = answer_reset['product'][sort_predict.index][:200]
    print('В данном регионе согласно предсказанию модели будет добыто', sort_answer.sum(), 'тысяч баррелей нефти')
    print('Что в итоге принесет чистую прибыль в размере:')
    return((sort_answer.sum() * price_for_kbarrel) - investition)

In [24]:
value_on_region(predicted_valid_1, target_valid_1)

В данном регионе согласно предсказанию модели будет добыто 29601.835651421894 тысяч баррелей нефти
Что в итоге принесет чистую прибыль в размере:


3320826.0431398526

In [25]:
value_on_region(predicted_valid_2, target_valid_2)

В данном регионе согласно предсказанию модели будет добыто 27589.081548181137 тысяч баррелей нефти
Что в итоге принесет чистую прибыль в размере:


2415086.6966815125

In [26]:
value_on_region(predicted_valid_3, target_valid_3)

В данном регионе согласно предсказанию модели будет добыто 28245.22214133296 тысяч баррелей нефти
Что в итоге принесет чистую прибыль в размере:


2710349.9635998327

По результатам нашего исследования мы видим, что на основе предсказаний модели каждый регион принесет хорошую прибыль, и самым прибыльным из них по результатам теста будет первый регион. Но это не дает нам возможности оценить риски и нормальность распределения, ведь при выборе скважин выбор будет всего из 500 точек, и для проверки этого мы можем использовать bootstrep.

## Расчёт прибыли и рисков 

Напишем функцию бутстрепа чтобы не переписывать ее для каждого региона, выведем 95% доверительные интервалы и посмотрим какие у нас риски потерпеть убытки в этих регионах.

In [27]:
def bootstrep_value(probabilities, target):
    values = []
    state = np.random.RandomState(12345)
    for i in range(1000):
        probabilities_to_series = pd.Series(probabilities)
        target_to_reset = target.reset_index().drop('index', axis = 1)
        subsample_target = target_to_reset.sample(n=500, replace=False, random_state=state)
        subsample_probabilities = probabilities_to_series[subsample_target.index]
        sort_predict = subsample_probabilities.sort_values(ascending = False)
        sort_answer = subsample_target['product'][sort_predict.index][:200]
        values.append((sort_answer.sum() * price_for_kbarrel) - investition)
    return(values)

In [28]:
values_1_region = bootstrep_value(predicted_valid_1, target_valid_1)

In [29]:
values_1_region = pd.DataFrame(values_1_region)
values_1_region.set_axis(['values'],axis = 'columns',inplace = True)
min_1 = values_1_region['values'].quantile(0.025)
max_1 = values_1_region['values'].quantile(0.975)
print('95% доверительный интервал прибыли находится в диапазоне от', min_1, 'до', max_1, 'тыс.руб.')

95% доверительный интервал прибыли находится в диапазоне от -126947.63803180121 до 879613.9678477967 тыс.руб.


In [30]:
print('Риск убытков составляет',
      values_1_region.query('values < 0')['values'].count() / values_1_region['values'].count() * 100, '%')

Риск убытков составляет 7.199999999999999 %


In [31]:
values_2_region = bootstrep_value(predicted_valid_2, target_valid_2)
values_2_region = pd.DataFrame(values_2_region)
values_2_region.set_axis(['values'],axis = 'columns',inplace = True)
min_2 = values_2_region['values'].quantile(0.025)
max_2 = values_2_region['values'].quantile(0.975)
print('95% доверительный интервал прибыли находится в диапазоне от', min_2, 'до', max_2, 'тыс.руб.')

95% доверительный интервал прибыли находится в диапазоне от 70899.38493535544 до 892985.24970006 тыс.руб.


In [32]:
print('Риск убытков составляет',
      values_2_region.query('values < 0')['values'].count() / values_2_region['values'].count() * 100, '%')

Риск убытков составляет 1.4000000000000001 %


In [33]:
values_3_region = bootstrep_value(predicted_valid_3, target_valid_3)
values_3_region = pd.DataFrame(values_3_region)
values_3_region.set_axis(['values'],axis = 'columns',inplace = True)
min_3 = values_3_region['values'].quantile(0.025)
max_3 = values_3_region['values'].quantile(0.975)
print('95% доверительный интервал прибыли находится в диапазоне от', min_3, 'до', max_3, 'тыс.руб.')

95% доверительный интервал прибыли находится в диапазоне от -143659.30684827152 до 963026.1544863078 тыс.руб.


In [34]:
print('Риск убытков составляет',
      values_3_region.query('values < 0')['values'].count() / values_3_region['values'].count() * 100, '%')

Риск убытков составляет 7.1 %


In [35]:
values_1_region.mean()

values    380710.890709
dtype: float64

In [36]:
values_2_region.mean()

values    448231.065148
dtype: float64

In [37]:
values_3_region.mean()

values    402796.587172
dtype: float64

Мы получили 95% доверительные интервали по всем трем регионам, и судя по результатам проверки бутстрепом у нас самые маленькие риски и самая высокая средняя прибыль во втором регионе, хотя когда мы просто брали среднее по всей выборке результат был прямо противоположный. Все было определенно не зря.

## Вывод:

Итого на основании проведенного исследования, можно сделать вывод что самые прибыльные регионы это второй и третий. В третьем регионе есть шанс заработать больше, а во втором все выглядит в разы безопаснее. Я считаю что в данной ситуации лучше выбрать второй регион, потому что в его доверительном интервале значительно меньше разброс, и почти в 5 раз меньше риски убытков, да и среднее у него выше. Мне кажется при таких прогнозах выбор очевиден.